In [ ]:
# Small Language Model(SLLM) 
# Step 01 - 가장 기초적인 문장 데이터셋 적용, 즉 개념 정의
# 데이터셋 -> 데이터로더 -> GPU 설정 -> SLLM 모델 정의 -> 학습

In [ ]:
# import 및 GPU 설정
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'Pytorch Version : {torch.__version__}, Device : {device}')

Pytorch Version : 2.7.1+cu118, Device : cuda


In [ ]:
# 데이터 토큰화 및 시퀀시 생성
# - Step 01 에서는 아주 간단한 텍스트 데이터를 사용해 토큰화하고, 시퀀시를 만들어 학습용으로 변환한다.

# 예시 텍스트 데이터
text = "hello world hello pytorch language model"

# 단어 단위로 토큰화
tokens = text.split() # split
# print(tokens)
vocab = list(set(tokens)) # 분리된 단어 -> 리스트
# print(vacab)
word2idx = { word:idx for idx, word in enumerate(vocab) } # 단어 리스트 -> 딕셔너리(value:key), index 값 출력
print(word2idx)
idx2word = { idx:word for word, idx in word2idx.items() } # 딕셔너리(key:value), word value 값 출력
# print(idx2word)

# 시퀀스 생성(입력: [hello, world], 출력: [world, hello] 등)
sequence_length = 2
data = []
for i in range(len(tokens) - sequence_length):
    input_seq = tokens[i:i+sequence_length]
    # print('input_seq : ', input_seq)
    target_seq = tokens[i+1:i+sequence_length+1]
    # print('target_seq : ', target_seq)
    data.append((
        [ word2idx[word] for word in input_seq ],
        [ word2idx[word] for word in target_seq ] 
    ))
print(data)

{'world': 0, 'language': 1, 'pytorch': 2, 'hello': 3, 'model': 4}
[([3, 0], [0, 3]), ([0, 3], [3, 2]), ([3, 2], [2, 1]), ([2, 1], [1, 4])]


In [55]:
# 데이터셋 및 데이터로더 생성
class TextDataset(Dataset):
    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        x, y = self.data[idx]
        return torch.tensor(x), torch.tensor(y)

# dataset
dataset = TextDataset(data)

# dataloader
dataloader = DataLoader(dataset=dataset, batch_size=2, shuffle=True)

In [ ]:
# SLLM 모델 정의
class SmallLanguageModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super(SmallLanguageModel, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.rnn = nn.GRU(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)
    
    def forward(self, x):
        x = self.embedding(x)
        output, _ = self.rnn(x)
        logits = self.fc(output)
        return logits

In [ ]:
# 학습
model = SmallLanguageModel(vacab_size=len(vocab), embed_dim=16, hidden_dim=32).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(100):
    for inputs, targets in dataloader:
        inputs, targets = inputs.to(device), targets.to(device)

        outputs = model(inputs)
        loss = criterion(outputs.view(-1, len(vocab)), targets.view(-1))

        # 오차역전파
        optimizer.zero_grad() # 미분 파리미터 초기화
        loss.backward() # 미분 연산
        optimizer.step() # 미분 연산 후 가중치,바이어스 파라미터 업데이트
    
    print(f'Epoch {epoch+1}, Loss : {loss.item():.4f}')

Epoch 1, Loss : 1.6555
Epoch 2, Loss : 1.4917
Epoch 3, Loss : 1.3273
Epoch 4, Loss : 1.0931
Epoch 5, Loss : 1.0393
Epoch 6, Loss : 0.9532
Epoch 7, Loss : 0.7608
Epoch 8, Loss : 0.4111
Epoch 9, Loss : 0.5966
Epoch 10, Loss : 0.5328
Epoch 11, Loss : 0.1902
Epoch 12, Loss : 0.3651
Epoch 13, Loss : 0.3194
Epoch 14, Loss : 0.3882
Epoch 15, Loss : 0.1814
Epoch 16, Loss : 0.2015
Epoch 17, Loss : 0.2410
Epoch 18, Loss : 0.3619
Epoch 19, Loss : 0.1777
Epoch 20, Loss : 0.1930
Epoch 21, Loss : 0.1942
Epoch 22, Loss : 0.1727
Epoch 23, Loss : 0.1748
Epoch 24, Loss : 0.0116
Epoch 25, Loss : 0.2124
Epoch 26, Loss : 0.2217
Epoch 27, Loss : 0.2240
Epoch 28, Loss : 0.1634
Epoch 29, Loss : 0.1956
Epoch 30, Loss : 0.0071
Epoch 31, Loss : 0.1814
Epoch 32, Loss : 0.2005
Epoch 33, Loss : 0.3533
Epoch 34, Loss : 0.3531
Epoch 35, Loss : 0.2091
Epoch 36, Loss : 0.1708
Epoch 37, Loss : 0.1916
Epoch 38, Loss : 0.3516
Epoch 39, Loss : 0.2073
Epoch 40, Loss : 0.1672
Epoch 41, Loss : 0.1822
Epoch 42, Loss : 0.1926
E